In [ ]:
from src.llm_plan.llm import GPT_OSS

system_prompt = "You are a useful assistant."
prompt = "Tell me a joke."

model = GPT_OSS(model_name="openai/gpt-oss-120b", map="cpu", reasoning="Low")

model.generate_sync(system_prompt, prompt)

In [ ]:
prompt = "What is the capital of France?"
model.generate_sync(system_prompt, prompt)

In [1]:
from google import genai

# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

response = client.models.generate_content(
    model="gemini-2.5-flash", contents="Explain how AI works in a few words"
)
print(response.text)

AI learns patterns from data to make smart decisions or predictions.
